In [1]:
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [2]:
# Loading the dataset into pandas data frame
path_data = "C:/Users/Rohan Mahesh Rao/Documents/PES1UG20EC156/Sem 6/ML/project/gtzan/Data/features_30_sec.csv"
data = pd.read_csv(path_data)
data = data.sample(frac=1)
data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
800,reggae.00000.wav,661794,0.329137,0.090313,0.094154,0.001773,2133.200382,276253.439506,2289.540104,130964.269614,...,67.178230,-9.199856,89.098007,3.286665,75.316483,-0.759790,86.541344,3.761663,104.204071,reggae
556,jazz.00056.wav,665280,0.372673,0.088457,0.198367,0.002145,2976.346758,269838.341104,3035.971529,63179.637699,...,41.537407,0.084078,33.646355,-2.027351,38.235390,-3.867052,62.623363,-2.114585,61.280365,jazz
133,classical.00033.wav,661794,0.304065,0.089344,0.012359,0.000187,1128.252612,124800.769894,1375.763705,82349.862182,...,90.259148,-1.891969,89.254494,-0.307660,63.502186,3.411614,70.433998,3.134486,84.851761,classical
922,rock.00022.wav,661794,0.415770,0.078892,0.173919,0.002272,2342.440836,256754.458502,2113.038593,71777.331607,...,53.345806,-7.464095,39.981232,2.070638,57.735928,-2.897590,40.805145,-0.363278,38.174831,rock
306,disco.00006.wav,661676,0.473975,0.077535,0.191986,0.002105,2848.348404,148665.474833,2490.612887,79205.359263,...,35.386063,-4.731374,38.574081,-2.610662,45.489277,-7.465981,52.653217,0.231177,43.680527,disco


In [3]:
xfeatures = ['chroma_stft_mean','rms_mean','spectral_centroid_mean','spectral_bandwidth_mean','tempo','mfcc1_mean']
labelnames = ['label']
#features = [[song_chroma_stft_meanval,rms_meanval,spectral_centroid_meanval,spectral_bandwidth_meanval,song_tempo[0],mfcc_meanval]]
metadata_pd = data[xfeatures]
label_pd = data['label']

# Converting the datatypes to numpy
metadata = metadata_pd.to_numpy()
label = label_pd.to_numpy()

# #Standardizing the dataset
# standardized_metadata = StandardScaler().fit_transform(metadata)

In [4]:
def extract_features(song_path):

    y, sr = librosa.load(song_path, duration=30) #returns audio time series and sample rate (y and sr respectively)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr) # Mel frequency cepstrum coefficients , we will take mfcc[1][1] i.e. the first coefficient
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    song_onset = librosa.onset.onset_strength(y=y , sr=sr)
    rms = librosa.feature.rms(y=y)
    song_chroma_stft = librosa.feature.chroma_stft(y=y,sr=sr)

    mfcc_meanval = np.mean(mfcc[1])
  
    song_chroma_stft_meanval = np.mean(song_chroma_stft)

    rms_meanval = np.mean(rms)

    spectral_centroid_meanval = np.mean(spectral_centroid)                          
    
    spectral_bandwidth_meanval = np.mean(spectral_bandwidth)

    song_tempo = librosa.beat.tempo(onset_envelope=song_onset, sr=sr)

    features = [[song_chroma_stft_meanval,rms_meanval,spectral_centroid_meanval,spectral_bandwidth_meanval,song_tempo[0],mfcc_meanval]]
    
    return features

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(metadata, label, test_size=0.2, random_state=42)

In [6]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [7]:
# Create a neural network model
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [8]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model
model.fit(X_train, y_train, epochs= 20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Rohan Mahesh Rao\anaconda3\lib\asyncio\base_ev

In [12]:

# Evaluate the performance of the model on the testing set
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 2ms/step
